In [1]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for outpu

In [3]:
import os
import cv2
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('TPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# Funciones para cargar las imagenes y annotaciones

Estas funciones tambien se encargaran de normalizar los puntos (dividiendo por /255) para asi que todos los valores esten en un mismo rango y nuestro modelo funcione mejor, ademas de redimensionar las imagenes a la resolucion compatible con el modelo (512x512)

In [5]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

train_images = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_1/train_1/*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (512,512)))
train_images = train_images.map(lambda x: x/255)

test_images = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_2/train_2/*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (512,512)))
test_images = test_images.map(lambda x: x/255)

val_images = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_3/train_3/*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (512,512)))
val_images = val_images.map(lambda x: x/255)

In [6]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

train_labels = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_1/annotation/*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

test_labels = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_2/annotation/*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

val_labels = tf.data.Dataset.list_files(r'/content/drive/MyDrive/EyePosition/dataset_format/train_3/annotation/*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))


In [7]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.batch(8)
train = train.prefetch(4)

test = tf.data.Dataset.zip((test_images, test_labels))
test = test.batch(8)
test = test.prefetch(4)

val = tf.data.Dataset.zip((val_images, val_labels))
val = val.batch(8)
val = val.prefetch(4)

In [8]:
data_samples = train.as_numpy_iterator()
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(50,50))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][0][idx]
    for i in range(0,79,2):
      cv2.circle(sample_image, [int(sample_coords[i]*512), int(sample_coords[i+1]*512)], 2, (255,0,0), -1)
    
    
    ax[idx].imshow(sample_image)

Output hidden; open in https://colab.research.google.com to view.

# Primer modelo (ResNet)

Para alcanzar nuestro objetivo vamos a usar un PreTrained Model para asi ahorrar un poco de tiempo a la hora de entrenar el modelo. En este caso se ha usado el modelo ResNet y se ha modificado para que sea compatible con el numero de puntos que tenemos de output en nuestro caso.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout
from tensorflow.keras.applications import ResNet152V2

model = Sequential([
    Input(shape=(512,512,3)), 
    ResNet152V2(include_top=False, input_shape=(512,512,3)),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(256, 3, 2, padding='same', activation='relu'),
    Conv2D(256, 2, 2, activation='relu'),
    Dropout(0.05),
    Conv2D(80, 2, 2),
    Conv2D(80, 2, 1),
    Reshape((80,))
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0007)
loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer, loss)

hist = model.fit(train, epochs=200, validation_data=val)

234545216/234545216 [==============================] - 1s 0us/step
Epoch 1/200
125/125 [==============================] - 403s 3s/step - loss: 0.0455 - val_loss: 0.1090
Epoch 2/200
125/125 [==============================] - 158s 1s/step - loss: 0.0078 - val_loss: 0.0257
Epoch 3/200
125/125 [==============================] - 156s 1s/step - loss: 0.0057 - val_loss: 0.0118
Epoch 4/200
125/125 [==============================] - 154s 1s/step - loss: 0.0029 - val_loss: 0.0027
Epoch 5/200
125/125 [==============================] - 155s 1s/step - loss: 0.0020 - val_loss: 0.0020
Epoch 6/200
125/125 [==============================] - 155s 1s/step - loss: 0.0021 - val_loss: 0.0018
Epoch 7/200
125/125 [==============================] - ETA: 0s - loss: 0.0018

In [ ]:
model.save('/content/drive/MyDrive/eyeposition.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/eyeposition.h5')
test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = model.predict(test_sample[0])
fig, ax = plt.subplots(ncols=3, figsize=(20,20))
for idx in range(3): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[idx]
    
    for i in range(0,79,2):
      cv2.circle(sample_image, [int(sample_coords[i]*512), int(sample_coords[i+1]*512)], 2, (255,0,0), -1)
    
    ax[idx].imshow(sample_image)

In [ ]:
model = load_model('/content/drive/MyDrive/eyeposition.h5')
hist = model.fit(train, epochs=100, validation_data=val)

Al comprobar el funcionamiento del modelo se ve que funciona correctamente, solo que es demasiado lento para nuestra aplicacion. Tras probar el modelo en tiempo real en mi ordenador personal se consigue una media de 2 segundos para procesar cada fotograma de video, es decir, 0.5 fps lo cual no es suficiente.

Como alternativa se puede intentar usar otro PreTrained Model que no sea ResNet. Accediendo a ModelZoo de TensorFlow se encuentra el Modelo "CenterNet MobileNetV2 FPN Keypoints 512x512" con las siguientes caracteristicas:


| Model Name | Speed (ms) | COCO mAP |
| --- | --- | --- |
| CenterNet MobileNetV2 FPN Keypoints 512x512 | 6 | 41.7 |

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model = load_model('/content/drive/MyDrive/eyeposition.h5')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    h,w,c = frame.shape
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_img, (512,512))
    
    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[0]
    
    for i in range(0,79,2):
        cv2.circle(frame, [int(sample_coords[i]*w), int(sample_coords[i+1]*h)], 2, (0,255,0), -1)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()